In [4]:
import os
import re
import pandas as pd
import cv2
#import ffmpeg
#from moviepy.editor import VideoFileClip

dir = r"\\fauad.fau.de\shares\ANKI\Projects\Swallowing\Data\from_Melda"
#dir = os.path.realpath('.')
dir

'\\\\fauad.fau.de\\shares\\ANKI\\Projects\\Swallowing\\Data\\from_Melda'

# Spreadsheet

In [5]:
df = pd.DataFrame(columns=['Patient ID', 'Video Name', 'Visit', 'Height', 'Width', 'if CRF', 'if Extracted', 'Quality' 'Comments'])

for folder in os.listdir(dir):
    folder_dir = os.path.join(dir, folder)
    #print(folder)
    if os.path.isdir(folder_dir) and folder.isdigit():
        for file in os.listdir(folder_dir):
            # skip if it's not a video
            if not file.endswith('.mp4'):
                continue

            # skip if it's an extracted segment
            # re to search for z.B. "Pt47_Visit1_1_fixed_001.mp4" "Pt47_Visit1_2_fixed_CRF8_001.mp4"
            pattern = r"_\d{3}\.mp4$"
            extract = re.search(pattern, file)
            if extract is not None:
                continue
            
            Patient_ID = int(folder)
            Video_Name = file
            
            # Visit: string, becaused of unknown meaning of "visit1_1"
            # re to search for "1" or "1_1" after "Visit" / before "_fixed"
            # inconsistent naming conventions: space between "Visit" and digit, inexistent "_fixed"
            pattern = r"Visit(\d+(_\d+)?)"
            visit = re.search(pattern, file)
            if visit is None:
                pattern = r"(\d+(_\d+)?)_fixed"
                visit = re.search(pattern, file)
            if visit is None:
                print(file)
                raise Exception("Error: Could not find visit number.")
            Visit = visit.group(1)
        
            # size: cv2 video properties
            cap = cv2.VideoCapture(os.path.join(dir, folder, file))
            if not cap.isOpened():
                raise Exception("Error: Could not open video.")
            else:
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                cap.release()
            Width = width
            Height = height
            
            # CRF: name check
            if 'CRF' in file:
                CRF = 1
            else:
                CRF = 0
            
            sc_file_path = os.path.join(folder_dir, file.replace('.mp4', '.sc'))
            if os.path.exists(sc_file_path):
                Extracted = 1
            else:
                Extracted = 0

            row = pd.DataFrame([[Patient_ID, Video_Name, Visit, Height, Width, CRF, Extracted, None, None]], 
                               columns=['Patient ID', 'Video Name', 'Visit', 'Height', "Width", 'if CRF', 'if Extracted', 'Quality' 'Comments'])
            df = pd.concat([df, row], ignore_index=True)


ValueError: 8 columns passed, passed data had 9 columns

In [24]:
df


,Patient ID,Video Name,Visit,Height,Width,if CRF,if Extracted
0,10000,Pt10000_Visit1_fixed.mp4,1,1108,1140,0,0
1,102,Pt102_Visit1_fixed.mp4,1,1108,1140,0,0
2,102,Pt102_Visit1_fixed_CRF8.mp4,1,1108,1140,1,0
3,102,Pt102_Visit2_fixed.mp4,2,1108,1140,0,0
4,102,Pt102_Visit2_fixed_CRF8.mp4,2,1108,1140,1,0
...,...,...,...,...,...,...,...
1115,996,Pt996_Visit1_fixed.mp4,1,1108,1140,0,0
1116,9972,Pt9972_Visit1_fixed.mp4,1,530,530,0,0
1117,9972,Pt9972_Visit2_fixed.mp4,2,1108,1140,0,0
1118,9972,Pt9972_Visit3_fixed.mp4,3,1108,1140,0,0


In [25]:
with open('from_melda.csv', 'w') as f:
    df.to_csv(f, index=False)
    

# Extraction Status Update

In [100]:
def find_sc_file(filename):
    sc_file = filename.replace('.mp4', '.sc')
    foldername = re.search(r"(\d+)_Visit", filename)
    if foldername is None:
        foldername = re.search(r"(\d+)_VIsit", filename)
    if foldername is None:
        foldername = re.search(r"(\d+)_visit", filename)
    foldername = foldername.group(1)
    if os.path.exists(os.path.join(dir, foldername, sc_file)):
        return 1
    else:
        return 0
    

In [102]:
df = pd.read_csv(os.path.join(dir, 'from_melda.csv'))
df['if Extracted'] = df['Video Name'].apply(find_sc_file)
df.to_csv('from_melda.csv', index=False)
    

# Track Extracted Events

In [2]:
df = pd.DataFrame(columns=['Patient ID', 'Video Name', 'Event Name', 'length', 'if annotated', 'swallow type', 'quality', 'comments'])
df_extracted = pd.read_csv(os.path.join(dir, 'from_melda_events.csv'))

for folder in os.listdir(dir):
    folder_dir = os.path.join(dir, folder)
    #print(folder)
    if os.path.isdir(folder_dir) and folder.isdigit():
        for file in os.listdir(folder_dir):
            
            # skip if it's not a video
            if not file.endswith('.mp4'):
                continue
            if file in df_extracted['Video Name'].values:
                continue

            # skip if it's an extracted segment
            # re to search for z.B. "Pt47_Visit1_1_fixed_001.mp4" "Pt47_Visit1_2_fixed_CRF8_001.mp4"
            pattern = r"_\d{3}\.mp4$"
            extract = re.search(pattern, file)
            if extract is None:
                continue
            
            Patient_ID = int(folder)
            Event_Name = file
            Video_Name = re.sub(pattern, '.mp4', file)
            
            
            clip = VideoFileClip(os.path.join(dir, folder, file))
            length = clip.duration


            # length: cv2 video properties
            #cap = cv2.VideoCapture(os.path.join(dir, folder, file))
            #if not cap.isOpened():
            #    raise Exception("Error: Could not open video.")
            #length = cap.get(cv2.CAP_PROP_POS_MSEC)
            #cap.release()

            row = pd.DataFrame([[Patient_ID, Video_Name, Event_Name, length, None, None, None, None]], 
                               columns=['Patient ID', 'Video Name', 'Event Name', 'length', 'if annotated', 'swallow type', 'quality', 'comments'])
            df = pd.concat([df, row], ignore_index=True)


In [3]:
df_extracted = pd.concat([df_extracted, df], ignore_index=True)
df_extracted.to_csv('from_melda_events.csv', index=False)